In [262]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Show plots in the notebook
%matplotlib inline

# Set the maximum amount of rows displayed at one time
from pandas import set_option
set_option('display.max_rows',20)

Loading BokehJS ...

In [22]:
# Load World Development Indicators (WDI) data from http://data.worldbank.org/data-catalog/world-development-indicators
data = pd.read_csv('../datasets/WDI_csv/WDIdata.csv',sep=',')
data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420022,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,30.1,NaN,30.5,NaN,NaN,33.5,32.4,NaN,NaN,NaN
420023,Zimbabwe,ZWE,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,57.4,58.2,58.4,58.6,58.8,58.9,58.9,59.0,59.1,NaN


In [259]:
# Locate World data for Capture Fisheries Production (metric tons) and Aquaculture production (metric tons)
world = data.loc[data['Country Name'] == 'World']
world_c = world.loc[world['Indicator Name'] == 'Capture fisheries production (metric tons)']
world_a = world.loc[world['Indicator Name'] == 'Aquaculture production (metric tons)']

# Cleaning data for graphing
world_cap = world_c.reset_index()
world_aq = world_a.reset_index()
capture = world_cap.iloc[:,5:61]
aquaculture = world_aq.iloc[:,5:61]
capture = capture.T
aquaculture = aquaculture.T

In [298]:
# Cleaning continued, and creating dataframe for graphing
world = pd.DataFrame(columns = ['Capture', 'Aquaculture'], index=capture.index)
world['Capture'] = capture
world['Aquaculture'] = aquaculture
world

,Capture,Aquaculture
1960,3.479018e+07,2.029179e+06
1961,3.863019e+07,1.948048e+06
1962,4.203568e+07,2.014404e+06
1963,4.299637e+07,2.247676e+06
1964,4.758683e+07,2.402677e+06
1965,4.865361e+07,2.604353e+06
1966,5.256864e+07,2.791746e+06
1967,5.602067e+07,2.915674e+06
1968,5.933623e+07,3.092313e+06
1969,5.790321e+07,3.238166e+06


In [9]:
world1 = world/1000000

In [515]:
from bokeh.models.annotations import Label
from bokeh.models import LabelSet
from bokeh.models.tools import HoverTool
from bokeh.models import NumeralTickFormatter, FuncTickFormatter
from bokeh.models import DataRange1d

In [12]:
# create a new plot with default tools, using figure
p = figure(plot_width=800, plot_height=600, 
           x_axis_label = 'Year', y_axis_label = 'Metric Tons', x_range = (1959, 2028),
           title = "Capture fisheries vs. aquaculture (farmed fish) production, World",
          toolbar_location = 'right')

p.xaxis.ticker = [1960, 1970, 1980, 1990, 2000, 2010, 2015]
p.yaxis.formatter = FuncTickFormatter(code="""
    return tick + " million "
""")
p.xgrid.visible = False


# Create source from World Data
source = ColumnDataSource(world1)

# Add Capture data
p.scatter('index', 'Capture', source = source, color = 'red', fill_alpha=1.0, size = 4)
a = p.scatter('index', 'Capture', source = source, color = 'darkred', fill_alpha=0, line_alpha = 0, hover_color = 'red', size = 10, hover_alpha = 1)
p.line('index', 'Capture', source = source, color = 'red', line_alpha=0.5, line_width = 1.5)

# Add Aquaculture data
p.scatter('index', 'Aquaculture', source = source, color = 'navy', fill_alpha=1.0, size = 4)
b = p.scatter('index', 'Aquaculture', source = source, color = 'navy', fill_alpha=0, line_alpha = 0, size = 10, hover_color = "navy")
p.line('index', 'Aquaculture', source = source, color = 'navy', line_alpha=1.0, line_width = 1.5)

# Add Hover Tool
# HoverTool.formatters = {'text'}
p.add_tools(HoverTool(tooltips=[("Capture Harvest", "@Capture{(0)} million metric tons in year @index")], 
                      renderers=[a], mode='vline'))
#p.add_tools(HoverTool(tooltips=None, renderers=[c], mode='hline'))
p.add_tools(HoverTool(tooltips= [("Aquaculture Harvest", "@Aquaculture{(0)} million in year @index")], renderers=[b], mode='vline'))

# Add Labels
label1 = Label(x=2015, y=106.004, x_offset=10, text="Aquaculture", text_color = 'navy', text_font_size='11pt')
label2 = Label(x=2015, y=93.737, x_offset=10, text="Wild Fisheries", text_color = 'red', text_font_size='11pt')

p.add_layout(label1)
p.add_layout(label2)

show(style(p)) # show the results

In [11]:
# Style function that takes in a plot
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '18pt'
    p.title.text_font = 'serif'
    
    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'
    
    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    return p

In [13]:
world.to_csv('fisheries.csv', sep=',')